In [ ]:
land_labels = ['agriculture', 'artisinal_mine', 'bare_ground', 'blooming', 'blow_down', 
               'conventional_mine', 'cultivation', 'habitation','primary', 'road',
               'selective_logging', 'slash_burn', 'water']

#train ground label with validation set
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import re
import random
import math

import vgg16_trainable as vgg16
import read_data
import utils
import csv

from sklearn.metrics import fbeta_score
from sklearn.metrics import accuracy_score

# The arguments y and p are both Nx17 numpy array arrays
# where N is the number of rows in the validation set.
# y is true_label, p is the train result

def optimise_f2_thresholds(y, p, thres, verbose=True, resolution=100):
    
    type_num = y.shape[1]
    def mf(x):
        p2 = np.zeros_like(p)
        for i in range(type_num):
            p2[:, i] = (p[:, i] > x[i]).astype(np.int)
        score = fbeta_score(y, p2, beta=2, average='samples')
        return score

    x = thres
    for i in range(type_num):
        best_i2 = 0
        best_score = 0
        for i2 in range(resolution):
            i2 = i2 / float(resolution)
            x[i] = i2
            score = mf(x)
            if score > best_score:
                best_i2 = i2
                best_score = score
        x[i] = best_i2
        if verbose:
            print(i, best_i2, best_score)
    return x

def f2_score(y_true, y_pred):
    # fbeta_score throws a confusing error if inputs are not numpy arrays
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # We need to use average='samples' here, any other average method will generate bogus results
    return fbeta_score(y_true, y_pred, beta=2, average='samples')

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

class Config():
    batch_size = 8
    steps = "-1"
    gpu = '/gpu:0'

    # checkpoint path and filename
    logdir = "./log"
    params_dir = "./params/"
    load_filename = params_dir + "vgg16_ground_dynamic.npy"
    save_filename = params_dir + "vgg16_ground_dynamic.npy"

    # path
    imgs_path = "./train-jpg/"
    labels_file = "./train_validation_v2_bin.csv"

    # iterations config
    max_iteration = 4000
    summary_iters = 50
    # refer to synset.txt for the order of labels
    # ground labels
    usecols = [1, 2, 3, 4, 5, 8, 9, 10, 13, 14, 15, 16, 17]

config = Config()
reader = read_data.Reader(config)

validation_config = Config()
validation_config.labels_file = "./validation_train_v2_bin.csv"
validation_reader = read_data.Reader(validation_config)

images = tf.placeholder(tf.float32, [None, 224, 224, 3])
true_out = tf.placeholder(tf.float32, [None, len(config.usecols)])
train_mode = tf.placeholder(tf.bool)

vgg = vgg16.Vgg16(config.load_filename, output_size=len(config.usecols))
vgg.build(images, train_mode)
print vgg.get_var_count() , "variables"
with tf.name_scope('loss'):
    cost = tf.reduce_mean(-tf.reduce_sum(true_out * tf.log(vgg.prob), [1]))
    tf.summary.scalar('loss', cost)
    valid_f2_score = 0
    tf.summary.scalar('validf2_score', valid_f2_score)
with tf.name_scope('train'):
    rate = 1e-3
    train = tf.train.GradientDescentOptimizer(rate).minimize(cost)
    tf.summary.scalar('learning_rate', rate)
    tf.summary.scalar('batch_size', config.batch_size)
    
    merged = tf.summary.merge_all()

with tf.device(config.gpu):    
    sess = tf.Session()
    writer = tf.summary.FileWriter(config.logdir, sess.graph)
    sess.run(tf.global_variables_initializer())

    print "start training"
    # start training
    # init thres
    thres = [0.2]*len(config.usecols)
    
    for idx in xrange(config.max_iteration):
        imgs, labels = reader.random_batch()
        # feed data into the model
        feed_dict = {
            images : imgs,
            true_out : labels,
            train_mode : True
        }
        sess.run(train, feed_dict=feed_dict)
        if  idx % 50 == 0:
            result = sess.run(merged, feed_dict=feed_dict)
            loss = sess.run(cost, feed_dict=feed_dict)
                        
            print idx, "cost:", loss
            writer.add_summary(result, idx)
            if idx % 250 == 0:
                valid_pred = []
                valid_true_out = []
                for x in  xrange(np.int32(np.ceil(4048/config.batch_size))):
                    valid_img, valid_label = validation_reader.batch()
                    valid_feed_dict = {
                        images : valid_img,
                        true_out: valid_label,
                        train_mode : False
                    }
                    valid_prob = sess.run(vgg.prob, feed_dict=valid_feed_dict)
                    valid_pred = np.append(valid_pred, valid_prob)
                    valid_true_out = np.append(valid_true_out, valid_label)
                valid_pred = np.reshape(valid_pred,[-1, len(config.usecols)])
                valid_true_out = np.reshape(valid_true_out, [-1, len(config.usecols)])
                
                valid_pred_out = np.zeros_like(valid_pred)
                for i in range(len(config.usecols)):
                    valid_pred_out[:, i] = (valid_pred[:, i] > thres[i]).astype(np.int)
                
                valid_f2_score = f2_score(valid_true_out, valid_pred_out)
                
                print "validation_f2_score:", valid_f2_score
                
                for i in xrange(13):
                    acy_score = accuracy_score(valid_true_out[:, i], valid_pred_out[:, i])
                    print "acy_score:\t", land_labels[i], "\t", acy_score
                
                thres = optimise_f2_thresholds(np.array(valid_true_out), np.array(valid_pred), thres, verbose=True, resolution=100)
                print "best thres:", thres
                
                vgg.save_npy(sess, config.save_filename)


conv5_1 (3, 3, 512, 512) (512,)
fc6 (25088, 4096) (4096,)
conv5_3 (3, 3, 512, 512) (512,)
conv5_2 (3, 3, 512, 512) (512,)
fc8 (4096, 1000) (1000,)
fc9 (1000, 13) (13,)
fc7 (4096, 4096) (4096,)
conv4_1 (3, 3, 256, 512) (512,)
conv4_2 (3, 3, 512, 512) (512,)
conv4_3 (3, 3, 512, 512) (512,)
conv3_3 (3, 3, 256, 256) (256,)
conv3_2 (3, 3, 256, 256) (256,)
conv3_1 (3, 3, 128, 256) (256,)
conv1_1 (3, 3, 3, 64) (64,)
conv1_2 (3, 3, 64, 64) (64,)
conv2_2 (3, 3, 128, 128) (128,)
conv2_1 (3, 3, 64, 128) (128,)
138370557 variables
start training
0 cost: 0.496851


/usr/local/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


validation_f2_score: 0.857942746722
acy_score:	agriculture 	0.871294466403
acy_score:	artisinal_mine 	0.991847826087
acy_score:	bare_ground 	0.977766798419
acy_score:	blooming 	0.987648221344
acy_score:	blow_down 	0.995800395257
acy_score:	conventional_mine 	0.996541501976
acy_score:	cultivation 	0.899950592885
acy_score:	habitation 	0.91847826087
acy_score:	primary 	0.932806324111
acy_score:	road 	0.928112648221
acy_score:	selective_logging 	0.994812252964
acy_score:	slash_burn 	0.993577075099
acy_score:	water 	0.888586956522
(0, 0.08, 0.85834550359865014)
(1, 0.03, 0.85919437042894076)
(2, 0.1, 0.85941208836728411)
(3, 0.07, 0.85950745852194277)
(4, 0.02, 0.85951191443054875)
(5, 0.02, 0.85987899639594223)
(6, 0.07, 0.86059368407360148)
(7, 0.05, 0.86986042858095025)


/usr/local/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


(8, 0.0, 0.8699051814021681)
(9, 0.07, 0.87301768329122786)
(10, 0.06, 0.87334004599933668)
(11, 0.05, 0.87334004599933668)
(12, 0.1, 0.87502126799079305)
best thres: [0.08, 0.03, 0.1, 0.07, 0.02, 0.02, 0.07, 0.05, 0.0, 0.07, 0.06, 0.05, 0.1]
('file saved', './params/vgg16_ground_dynamic.npy')
50 cost: 1.80969
100 cost: 1.39635
150 cost: 2.87075
200 cost: 0.46947
250 cost: 1.53038
validation_f2_score: 0.865184100646
acy_score:	agriculture 	0.845849802372
acy_score:	artisinal_mine 	0.977766798419
acy_score:	bare_ground 	0.963191699605
acy_score:	blooming 	0.992835968379
acy_score:	blow_down 	0.994812252964
acy_score:	conventional_mine 	0.996788537549
acy_score:	cultivation 	0.877964426877
acy_score:	habitation 	0.868330039526
acy_score:	primary 	0.933053359684
acy_score:	road 	0.856719367589
acy_score:	selective_logging 	0.990859683794
acy_score:	slash_burn 	0.993577075099
acy_score:	water 	0.810523715415
(0, 0.1, 0.86628713515680167)
(1, 0.07, 0.86653315584486423)
(2, 0.05, 0.867255350

In [ ]:
# rare class
import os
from ground_train import g_train
class Config():
    batch_size = 8
    steps = "-1"
    gpu = '/gpu:0'

    # checkpoint path and filename
    logdir = "./log"
    params_dir = "./params/"
    load_filename = "vgg16.npy"
    save_filename = params_dir + "vgg16_ground_rare.npy"

    # path
    imgs_path = "./train-jpg/"
    labels_file = "./rare_train_validation_v2_bin.csv"
    
    # iterations config
    max_iteration = 8000
    summary_iters = 50
    valid_iters = 250
    # refer to synset.txt for the order of labels
    # ground labels
    usecols = [2, 3, 4, 5, 8, 15, 16]

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

config = Config()
valid_config = Config()
valid_config.labels_file = "./rare_validation_train_v2_bin.csv"
g_train(config, valid_config)

conv5_1 (3, 3, 512, 512) (512,)
fc6 (25088, 4096) (4096,)
conv5_3 (3, 3, 512, 512) (512,)
fc7 (4096, 4096) (4096,)
fc8 (4096, 1000) (1000,)
conv5_2 (3, 3, 512, 512) (512,)
conv4_1 (3, 3, 256, 512) (512,)
conv4_2 (3, 3, 512, 512) (512,)
conv4_3 (3, 3, 512, 512) (512,)
conv3_3 (3, 3, 256, 256) (256,)
conv3_2 (3, 3, 256, 256) (256,)
conv3_1 (3, 3, 128, 256) (256,)
conv1_1 (3, 3, 3, 64) (64,)
conv1_2 (3, 3, 64, 64) (64,)
conv2_2 (3, 3, 128, 128) (128,)
conv2_1 (3, 3, 64, 128) (128,)
138364551 variables
start training
0 cost: 1.75749


/usr/local/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


validation_f2_score: 0.0
acy_score:	artisinal_mine 	0.83746898263
acy_score:	bare_ground 	0.600992555831
acy_score:	blooming 	0.85459057072
acy_score:	blow_down 	0.954838709677
acy_score:	conventional_mine 	0.955831265509
acy_score:	selective_logging 	0.849131513648
acy_score:	slash_burn 	0.902729528536
(0, 0.0, 0.15392038344891942)
(1, 0.0, 0.4502008743944228)
(2, 0.14, 0.52288141965561319)
(3, 0.14, 0.52288141965561319)
(4, 0.13, 0.52553937997486389)
(5, 0.15, 0.56822834251494303)
(6, 0.15, 0.57735335728511905)
best thres: [0.0, 0.0, 0.14, 0.14, 0.13, 0.15, 0.15]
('file saved', './params/vgg16_ground_rare.npy')
50 cost: 0.745256
100 cost: 0.618107
150 cost: 0.683303
200 cost: 0.1269
250 cost: 0.638613
validation_f2_score: 0.645401580416
acy_score:	artisinal_mine 	0.164015904573
acy_score:	bare_ground 	0.403081510934
acy_score:	blooming 	0.869532803181
acy_score:	blow_down 	0.924453280318
acy_score:	conventional_mine 	0.954274353877
acy_score:	selective_logging 	0.524850894632
acy_sco

In [ ]:
# all 17 classes
import os
from ground_train import g_train
class Config():
    batch_size = 16
    steps = "-1"
    gpu = '/gpu:0'

    # checkpoint path and filename
    logdir = "./log"
    params_dir = "./params/"
    load_filename = params_dir + "vgg16_all.npy"
    save_filename = params_dir + "vgg16_all_.npy"

    # path
    imgs_path = "./train-jpg/"
    labels_file = "./train_validation_v2_bin.csv"
    
    # iterations config
    max_iteration = 8000
    summary_iters = 50
    valid_iters = 250
    # refer to synset.txt for the order of labels
    # ground labels
    usecols = range(1,18)

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

config = Config()
valid_config = Config()
valid_config.labels_file = "./validation_train_v2_bin.csv"
thres = [0.08, 0.03, 0.03, 0.05, 0.02, 0.06, 0.11, 0.01, 0.06, 0.06, 0.06, 0.06, 0.1, 0.05, 0.06, 0.02, 0.07]
g_train(config, valid_config, thres)

conv5_1 (3, 3, 512, 512) (512,)
fc6 (25088, 4096) (4096,)
conv5_3 (3, 3, 512, 512) (512,)
conv5_2 (3, 3, 512, 512) (512,)
fc8 (4096, 1000) (1000,)
fc9 (1000, 17) (17,)
fc7 (4096, 4096) (4096,)
conv4_1 (3, 3, 256, 512) (512,)
conv4_2 (3, 3, 512, 512) (512,)
conv4_3 (3, 3, 512, 512) (512,)
conv3_3 (3, 3, 256, 256) (256,)
conv3_2 (3, 3, 256, 256) (256,)
conv3_1 (3, 3, 128, 256) (256,)
conv1_1 (3, 3, 3, 64) (64,)
conv1_2 (3, 3, 64, 64) (64,)
conv2_2 (3, 3, 128, 128) (128,)
conv2_1 (3, 3, 64, 128) (128,)
138374561 variables
start training
0 cost: 2.64552


/usr/local/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


validation_f2_score: 0.786839175546
acy_score:	agriculture 	0.815217391304
acy_score:	artisinal_mine 	0.992341897233
acy_score:	bare_ground 	0.977766798419
acy_score:	blooming 	0.992835968379
acy_score:	blow_down 	0.995800395257
acy_score:	clear 	0.867835968379
acy_score:	cloudy 	0.981966403162
acy_score:	conventional_mine 	0.996541501976
acy_score:	cultivation 	0.900938735178
acy_score:	habitation 	0.9145256917
acy_score:	haze 	0.958003952569
acy_score:	partly_cloudy 	0.965909090909
acy_score:	primary 	0.91847826087
acy_score:	road 	0.827075098814
acy_score:	selective_logging 	0.995059288538
acy_score:	slash_burn 	0.993577075099
acy_score:	water 	0.8979743083
(0, 0.06, 0.81961815160854756)
(1, 0.03, 0.82055628474225151)
(2, 0.02, 0.82162050322182878)
(3, 0.09, 0.82174990280774995)
(4, 0.01, 0.82175043639907996)
(5, 0.06, 0.84754820691225974)
(6, 0.05, 0.84930189193130179)
(7, 0.01, 0.8496811572436167)
(8, 0.04, 0.85892670845163677)
(9, 0.04, 0.86847414891988828)
(10, 0.04, 0.871195644

In [1]:
# all 17 classes including trainning validation set
import os
from ground_train import g_train
class Config():
    batch_size = 16
    steps = "-1"
    gpu = '/gpu:0'

    # checkpoint path and filename
    logdir = "./log"
    params_dir = "./params/"
    load_filename = params_dir + "vgg16_all.npy"
    save_filename = params_dir + "vgg16_all.npy"

    # path
    imgs_path = "./train-jpg/"
    labels_file = "./train_v2_bin.csv"
    
    # iterations config
    max_iteration = 501
    summary_iters = 50
    valid_iters = 250
    # refer to synset.txt for the order of labels
    # ground labels
    usecols = range(1,18)

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

config = Config()
valid_config = Config()
valid_config.labels_file = "./validation_train_v2_bin.csv"
thres = [0.08, 0.03, 0.03, 0.05, 0.02, 0.06, 0.11, 0.01, 0.06, 0.06, 0.06, 0.06, 0.1, 0.05, 0.06, 0.02, 0.07]
g_train(config, valid_config, thres)

conv5_1 (3, 3, 512, 512) (512,)
fc6 (25088, 4096) (4096,)
conv5_3 (3, 3, 512, 512) (512,)
conv5_2 (3, 3, 512, 512) (512,)
fc8 (4096, 1000) (1000,)
fc9 (1000, 17) (17,)
fc7 (4096, 4096) (4096,)
conv4_1 (3, 3, 256, 512) (512,)
conv4_2 (3, 3, 512, 512) (512,)
conv4_3 (3, 3, 512, 512) (512,)
conv3_3 (3, 3, 256, 256) (256,)
conv3_2 (3, 3, 256, 256) (256,)
conv3_1 (3, 3, 128, 256) (256,)
conv1_1 (3, 3, 3, 64) (64,)
conv1_2 (3, 3, 64, 64) (64,)
conv2_2 (3, 3, 128, 128) (128,)
conv2_1 (3, 3, 64, 128) (128,)
138374561 variables
start training
0 cost: 4.08928
validation_f2_score: 0.909181579401
acy_score:	agriculture 	0.888339920949
acy_score:	artisinal_mine 	0.993577075099
acy_score:	bare_ground 	0.949110671937
acy_score:	blooming 	0.984436758893
acy_score:	blow_down 	0.995553359684
acy_score:	clear 	0.937005928854
acy_score:	cloudy 	0.981966403162
acy_score:	conventional_mine 	0.990118577075
acy_score:	cultivation 	0.876235177866
acy_score:	habitation 	0.940711462451
acy_score:	haze 	0.9256422

/usr/local/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


(6, 0.01, 0.91351567039289272)
(7, 0.01, 0.91351567039289272)
(8, 0.06, 0.91351567039289272)
(9, 0.04, 0.91372659465509332)
(10, 0.08, 0.91394631296877349)
(11, 0.07, 0.91403056652077774)
(12, 0.08, 0.91417448576685689)
(13, 0.06, 0.91442256789346332)
(14, 0.13, 0.91490649168412341)
(15, 0.05, 0.91527801686833721)
(16, 0.03, 0.91813899396714971)
best thres: [0.08, 0.03, 0.06, 0.1, 0.04, 0.05, 0.01, 0.01, 0.06, 0.04, 0.08, 0.07, 0.08, 0.06, 0.13, 0.05, 0.03]
('file saved', './params/vgg16_all.npy')
50 cost: 3.79549


KeyboardInterrupt: 

In [1]:
# all 17 classes fix cloudy search
import os
from ground_train import g_train
class Config():
    batch_size = 16
    steps = "-1"
    gpu = '/gpu:0'

    # checkpoint path and filename
    logdir = "./log"
    params_dir = "./params/"
    load_filename = params_dir + "vgg16_all_2.npy"
    save_filename = params_dir + "vgg16_all_2.npy"

    # path
    imgs_path = "./train-jpg/"
    labels_file = "./train_validation_v2_bin.csv"
    
    # iterations config
    max_iteration = 8000
    summary_iters = 50
    valid_iters = 250
    # refer to synset.txt for the order of labels
    # ground labels
    usecols = range(1,18)

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

config = Config()
valid_config = Config()
valid_config.labels_file = "./validation_train_v2_bin.csv"
thres = [0.08, 0.03, 0.03, 0.05, 0.02, 0.06, 0.11, 0.01, 0.06, 0.06, 0.06, 0.06, 0.1, 0.05, 0.06, 0.02, 0.07]
g_train(config, valid_config, thres)

conv5_1 (3, 3, 512, 512) (512,)
fc6 (25088, 4096) (4096,)
conv5_3 (3, 3, 512, 512) (512,)
conv5_2 (3, 3, 512, 512) (512,)
fc8 (4096, 1000) (1000,)
fc9 (1000, 17) (17,)
fc7 (4096, 4096) (4096,)
conv4_1 (3, 3, 256, 512) (512,)
conv4_2 (3, 3, 512, 512) (512,)
conv4_3 (3, 3, 512, 512) (512,)
conv3_3 (3, 3, 256, 256) (256,)
conv3_2 (3, 3, 256, 256) (256,)
conv3_1 (3, 3, 128, 256) (256,)
conv1_1 (3, 3, 3, 64) (64,)
conv1_2 (3, 3, 64, 64) (64,)
conv2_2 (3, 3, 128, 128) (128,)
conv2_1 (3, 3, 64, 128) (128,)
138374561 variables
start training
0 cost: 2.86239
validation_f2_score: 0.920365654135
acy_score:	agriculture 	0.90192687747
acy_score:	artisinal_mine 	0.990365612648
acy_score:	bare_ground 	0.954051383399
acy_score:	blooming 	0.991106719368
acy_score:	blow_down 	0.994812252964
acy_score:	clear 	0.940711462451
acy_score:	cloudy 	0.978507905138
acy_score:	conventional_mine 	0.996788537549
acy_score:	cultivation 	0.87104743083
acy_score:	habitation 	0.923418972332
acy_score:	haze 	0.952816205

/usr/local/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


(6, 0.55, 0.91394259721544802)
(7, 0.01, 0.91394259721544802)
(8, 0.06, 0.91394259721544802)
(9, 0.07, 0.91415958631638172)
(10, 0.04, 0.91420205456232129)
(11, 0.07, 0.91427531896510672)
(12, 0.0, 0.91442043941520412)
(13, 0.07, 0.9145018355082315)
(14, 0.15, 0.91457765168422778)
(15, 0.06, 0.9160261644575366)
(16, 0.09, 0.91624672733189427)
best thres: [0.05, 0.04, 0.04, 0.05, 0.04, 0.04, 0.55, 0.01, 0.06, 0.07, 0.04, 0.07, 0.0, 0.07, 0.15, 0.06, 0.09]
('file saved', './params/vgg16_all_2.npy')
50 cost: 2.51706
100 cost: 4.81528
150 cost: 4.18101
200 cost: 4.45183
250 cost: 3.73199
validation_f2_score: 0.912373643341
acy_score:	agriculture 	0.842391304348
acy_score:	artisinal_mine 	0.995059288538
acy_score:	bare_ground 	0.958250988142
acy_score:	blooming 	0.968626482213
acy_score:	blow_down 	0.993083003953
acy_score:	clear 	0.922183794466
acy_score:	cloudy 	0.984436758893
acy_score:	conventional_mine 	0.99604743083
acy_score:	cultivation 	0.876235177866
acy_score:	habitation 	0.94861

KeyboardInterrupt: 

In [ ]:
# all 17 classes fix cloudy search
import os
from ground_train import g_train
class Config():
    batch_size = 16
    steps = "-1"
    gpu = '/gpu:0'

    # checkpoint path and filename
    logdir = "./log"
    params_dir = "./params/"
    load_filename = params_dir + "vgg16_all_2.npy"
    save_filename = params_dir + "vgg16_all_2_v.npy"

    # path
    imgs_path = "./train-jpg/"
    labels_file = "./validation_train_v2_bin.csv"
    
    # iterations config
    max_iteration = 8000
    summary_iters = 50
    valid_iters = 250
    # refer to synset.txt for the order of labels
    # ground labels
    usecols = range(1,18)

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

config = Config()
valid_config = Config()
valid_config.labels_file = "./validation_train_v2_bin.csv"
thres = [0.07, 0.01, 0.07, 0.05, 0.05, 0.06, 0.48, 0.01, 0.05, 0.05, 0.09, 0.04, 0.08, 0.08, 0.04, 0.02, 0.06]
g_train(config, valid_config, thres)

conv5_1 (3, 3, 512, 512) (512,)
fc6 (25088, 4096) (4096,)
conv5_3 (3, 3, 512, 512) (512,)
conv5_2 (3, 3, 512, 512) (512,)
fc8 (4096, 1000) (1000,)
fc9 (1000, 17) (17,)
fc7 (4096, 4096) (4096,)
conv4_1 (3, 3, 256, 512) (512,)
conv4_2 (3, 3, 512, 512) (512,)
conv4_3 (3, 3, 512, 512) (512,)
conv3_3 (3, 3, 256, 256) (256,)
conv3_2 (3, 3, 256, 256) (256,)
conv3_1 (3, 3, 128, 256) (256,)
conv1_1 (3, 3, 3, 64) (64,)
conv1_2 (3, 3, 64, 64) (64,)
conv2_2 (3, 3, 128, 128) (128,)
conv2_1 (3, 3, 64, 128) (128,)
138374561 variables
start training
0 cost: 4.02532
validation_f2_score: 0.91476085334
acy_score:	agriculture 	0.889328063241
acy_score:	artisinal_mine 	0.989871541502
acy_score:	bare_ground 	0.972579051383
acy_score:	blooming 	0.992835968379
acy_score:	blow_down 	0.995800395257
acy_score:	clear 	0.935770750988
acy_score:	cloudy 	0.980484189723
acy_score:	conventional_mine 	0.996788537549
acy_score:	cultivation 	0.865859683794
acy_score:	habitation 	0.946146245059
acy_score:	haze 	0.94688735

/usr/local/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


(6, 0.17, 0.91930555509148859)
(7, 0.01, 0.91930555509148859)
(8, 0.05, 0.91930555509148859)
(9, 0.04, 0.91964417744936899)
(10, 0.09, 0.91964417744936899)
(11, 0.04, 0.91964417744936899)
(12, 0.0, 0.91966976615982021)
(13, 0.08, 0.91966976615982021)
(14, 0.09, 0.91991745832418137)
(15, 0.03, 0.91994701482692176)
(16, 0.06, 0.91994701482692176)
best thres: [0.08, 0.01, 0.1, 0.08, 0.04, 0.06, 0.17, 0.01, 0.05, 0.04, 0.09, 0.04, 0.0, 0.08, 0.09, 0.03, 0.06]
('file saved', './params/vgg16_all_2_v.npy')
50 cost: 3.55867
100 cost: 2.2593
150 cost: 3.52749
200 cost: 4.09793
250 cost: 2.82969
validation_f2_score: 0.922468268316
acy_score:	agriculture 	0.900444664032
acy_score:	artisinal_mine 	0.97233201581
acy_score:	bare_ground 	0.98023715415
acy_score:	blooming 	0.992341897233
acy_score:	blow_down 	0.995800395257
acy_score:	clear 	0.942440711462
acy_score:	cloudy 	0.986166007905
acy_score:	conventional_mine 	0.988389328063
acy_score:	cultivation 	0.891057312253
acy_score:	habitation 	0.9192